In [ ]:
import os

import scanpy as sc

from gears import GEARS, PertData

Load "norman" data:

In [ ]:
norman = PertData(data_path="./data")
norman.load(data_name="norman")

Load "ReplogleNorman2020_E7" data.
(This data needs to be generated by executing "preprocessing/ReplogleNorman2020_E7.ipynb".)

In [ ]:
dataset_name = "ReplogleNorman2020_E7"
adata = sc.read_h5ad(
    filename=os.path.join("preprocessing", f"{dataset_name}", "adata.h5ad")
)
ReplogleNorman2020_E7 = PertData(data_path="./data")
ReplogleNorman2020_E7.new_data_process(dataset_name=dataset_name, adata=adata)
ReplogleNorman2020_E7.load(data_path=f"./data/{dataset_name}")

Split data and get dataloaders:

In [ ]:
# Specify data split.
norman.prepare_split(split="single", seed=42)

# Get dataloader.
norman.get_dataloader(batch_size=32, test_batch_size=128)

Set up and train GEARS model:

In [ ]:
gears_model = GEARS(pert_data=norman, device="cpu")
gears_model.model_initialize()
gears_model.train()

Save or load the model:

In [ ]:
gears_model.save_model(path="gears")
gears_model.load_pretrained(path="gears")

Predict:

In [ ]:
predictions, _ = gears_model.predict(pert_list=[["CBL", "CNN1"], ["FEV"]])